In [4]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
from pathlib import Path
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.object_segmentation.controller import SegmentationController
from multiplex_pipeline.object_segmentation.segmenters import Cellpose4Segmenter, InstansegSegmenter
import multiplex_pipeline.object_segmentation.cleaners as cleaners
from multiplex_pipeline.object_segmentation.preprocessing_controller import PreSegmentationProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load analysis settings

In [ ]:
# load analysis configuration
settings_path = r'C:\BLCA-1_Analysis\analysis_settings_BLCA1.yaml'

overwrite_mask = True

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-1/BLCA-1_Final',
 'analysis_name': 'BLCA-1_Analysis',
 'local_analysis_dir': 'R:/Wayne/BLCA',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/logs'),
 'detection_image': 'BLCA-1_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/temp'),
 'cores_dir_output': WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': '

### Define the logger

In [6]:
log_file = settings['log_dir'] / f"cores_segmentation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [7]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_000.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_001.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_002.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_003.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_004.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_005.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_006.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_007.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_008.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_009.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_010.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_011.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_012.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_013.zarr'),
 WindowsPath('R:/Wayne/BLCA/BLCA-1_Analysis/cores/Core_014.zar

### Setup

In [ ]:
# create requested elements
channels = settings['segmentation']['input']
print(f'Segmentation input: {channels}')

# define pre-processor
preseg_processor = PreSegmentationProcessor(normalize = settings['segmentation']['preprocessing']['normalize'],
                                            denoise = settings['segmentation']['preprocessing']['denoise'],
                                            mix = settings['segmentation']['preprocessing']['mix']
                                            )

# define segmenter
if settings['segmentation']['package'] == 'instanseg':
    model = settings['segmentation']['model']
    segmenter = InstansegSegmenter(model_type = model, **settings['segmentation']['kwargs'])
else:
    raise ValueError(f"Segmentation package {settings['segmentation']['package']} to be implemented.")

# define cleaner
cleaner_name = settings['segmentation']['cleaner']
cleaner_class = getattr(cleaners, cleaner_name)
cleaner = cleaner_class()

# define controller 
segmentation_controller = SegmentationController(segmenter, cleaner = cleaner, resolution_level = settings['segmentation']['resolution_level'], overwrite_mask = overwrite_mask)

# channels used for segmentation
input_channels = [f'{ch}_preseg' for ch in channels]

Segmentation input: ['DAPI', 'HLA1', 'CD44', 'NaKATPase', 'CD11C', 'CD45', 'pCK26', 'pS6', 'HES1']
Model fluorescence_nuclei_and_cells version 0.1.0 already downloaded in c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda
2025-10-08 14:02:02.218 | INFO     | multiplex_pipeline.object_segmentation.controller:__init__:30 - Initialized SegmentationController with segmenter=<multiplex_pipeline.object_segmentation.segmenters.InstansegSegmenter object at 0x000001F8747C2AD0>, resolution_level=0, pyramid_levels=3, downscale=2, cleaner=<multiplex_pipeline.object_segmentation.cleaners.BlobCleaner object at 0x000001F875DF5BD0> and overwrite_mask=True


### Segmentation

In [9]:
for sd_path in path_list[:1]:
    
    print(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)
    
    # run preprocessing
    sdata = preseg_processor.run(sdata, channels = channels)

    # run segmentation
    segmentation_controller.run(sdata, channels = input_channels, mask_name = settings['segmentation']['output_name'])

Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

2025-10-08 14:03:52.245 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with percentiles [1, 99.5].
2025-10-08 14:03:54.278 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with percentiles [1, 99.5].
2025-10-08 14:03:56.193 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with percentiles [1, 99.5].
2025-10-08 14:03:58.197 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with percentiles [1, 99.5].
2025-10-08 14:03:59.910 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with percentiles [1, 99.5].
2025-10-08 14:04:02.343 | INFO     | multiplex_pipeline.object_segmentation.preprocessing_controller:run_normalize:95 - Applied normalization with per

### Sneak peak

In [15]:
# refresh the object
sdata_org = sd.read_zarr(path_list[4])
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

SpatialData object, with associated Zarr store: C:\BLCA-1_Analysis\cores\Core_004.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD3': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD8a': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD11C': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD11b': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD20': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD31': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD44': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD45': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD45RO': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD68': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD127': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD163': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CDC6': DataTree[cyx] (1, 7040, 7232), (1

In [16]:
Interactive(sdata_org)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-09-24 10:38:40.208 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:40.208 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:50.523 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:50.531 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:39:07.940 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:39:07.948 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
